## Libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
import warnings

2024-08-03 14:31:27.935633: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 14:31:27.935742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 14:31:28.064981: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Dateset and Parameters

In [2]:
base_dir = "/kaggle/input/brain-mri/dataset"
test_dir = os.path.join(base_dir, 'testing')
train_dir = os.path.join(base_dir, 'training')
categories = ['glioma', 'meningioma', 'notumor', 'pituitary']

img_size = 128
batch_size = 16
epochs = 50

## Data Preparation

In [3]:
def prepare_data(directory, categories, img_size):
    data = []
    labels = []
    for category in categories:
        path = os.path.join(directory, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = tf.keras.preprocessing.image.load_img(os.path.join(path, img), target_size=(img_size, img_size), color_mode="grayscale")
                img_array = tf.keras.preprocessing.image.img_to_array(img_array)
                img_array = img_array / 255.0
                data.append(img_array)
                labels.append(class_num)
            except Exception as e:
                print(f"Failed to read image: {os.path.join(path, img)}, error: {e}")  # Debug statement
    return np.array(data), np.array(labels)

# Load and preprocess data
X_train, y_train = prepare_data(train_dir, categories, img_size)
X_test, y_test = prepare_data(test_dir, categories, img_size)
y_train = to_categorical(y_train, num_classes=len(categories))
y_test = to_categorical(y_test, num_classes=len(categories))

## Load and preprocess data

In [4]:
print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

Training samples: 5711
Testing samples: 1311


In [5]:
print(f"Training class distribution: {Counter(np.argmax(y_train, axis=1))}")
print(f"Testing class distribution: {Counter(np.argmax(y_test, axis=1))}")

Training class distribution: Counter({2: 1595, 3: 1457, 1: 1339, 0: 1320})
Testing class distribution: Counter({2: 405, 1: 306, 0: 300, 3: 300})


## Data Augmentation

In [6]:
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)

### Capsule Network Model

In [7]:
class CapsuleLayer(layers.Layer):
    def __init__(self, num_capsules, dim_capsules, routings=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.dim_capsules = dim_capsules
        self.routings = routings
        self.kernel = None

    def build(self, input_shape):
        assert len(input_shape) == 3, "Input shape should be [None, input_capsule_num, input_capsule_dim]"
        self.input_capsule_num = input_shape[1]
        self.input_capsule_dim = input_shape[2]
        self.kernel = self.add_weight(shape=(self.input_capsule_num, self.num_capsules, self.input_capsule_dim, self.dim_capsules),
                                      initializer='glorot_uniform',
                                      name='capsule_kernel')

    def call(self, inputs, training=None):
        inputs_expand = tf.expand_dims(tf.expand_dims(inputs, 2), -1)
        inputs_tiled = tf.tile(inputs_expand, [1, 1, self.num_capsules, 1, 1])
        inputs_hat = tf.reduce_sum(inputs_tiled * self.kernel, axis=3)

        b = tf.zeros(shape=[tf.shape(inputs)[0], self.input_capsule_num, self.num_capsules, 1])
        for i in range(self.routings):
            c = tf.nn.softmax(b, axis=2)
            outputs = tf.reduce_sum(c * inputs_hat, axis=1, keepdims=True)
            outputs = tf.squeeze(tf.nn.l2_normalize(outputs, -1), axis=1)
            if i < self.routings - 1:
                b += tf.reduce_sum(inputs_hat * tf.expand_dims(outputs, 1), -1, keepdims=True)

        return outputs

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_capsules, self.dim_capsules)

class Length(layers.Layer):
    def call(self, inputs, **kwargs):
        return tf.sqrt(tf.reduce_sum(tf.square(inputs), -1) + 1e-9)

### Define the CapsNet model

In [8]:
input_image = layers.Input(shape=(img_size, img_size, 1))
conv1 = layers.Conv2D(128, (9, 9), activation='relu')(input_image)
conv2 = layers.Conv2D(128, (9, 9), strides=(2, 2), activation='relu')(conv1)
conv2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Dropout(0.3)(conv2)
primary_caps = layers.Conv2D(32 * 4, (9, 9), activation='relu')(conv2)
primary_caps_reshape = layers.Reshape((-1, 128))(primary_caps)

print("Shape of primary_caps:", primary_caps.shape)

Shape of primary_caps: (None, 20, 20, 128)


### Adjust reshape layer based on printed shape


In [9]:
capsule = CapsuleLayer(num_capsules=len(categories), dim_capsules=16)(primary_caps_reshape)
output = Length()(capsule)

model = models.Model(inputs=input_image, outputs=output)
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 120, 120, 128)  │        10,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 56, 56, 128)    │     1,327,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 20, 20, 128)    │     1,327,232 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 400, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ capsule_layer (CapsuleLayer)    │ (None, 4, 16)          │     3,276,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ length (Length)                 │ (None, 4)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,942,272 (22.67 MB)

 Trainable params: 5,942,016 (22.67 MB)

 Non-trainable params: 256 (1.00 KB)

In [10]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [11]:
class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

In [12]:
warnings.filterwarnings('ignore', category=UserWarning, module='keras.src.trainers.data_adapters.py_dataset_adapter')

## Training the Model

In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    class_weight=class_weights
)

Epoch 1/50
  1/356 ━━━━━━━━━━━━━━━━━━━━ 1:53:32 19s/step - accuracy: 0.3750 - loss: 1.4117

I0000 00:00:1722695566.449863      73 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


356/356 ━━━━━━━━━━━━━━━━━━━━ 90s 199ms/step - accuracy: 0.2450 - loss: 1.3857 - val_accuracy: 0.2449 - val_loss: 1.3863
Epoch 2/50
  1/356 ━━━━━━━━━━━━━━━━━━━━ 29s 83ms/step - accuracy: 0.1875 - loss: 1.4016

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


356/356 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.1875 - loss: 1.4016 - val_accuracy: 0.2387 - val_loss: 1.3863
Epoch 3/50
356/356 ━━━━━━━━━━━━━━━━━━━━ 35s 97ms/step - accuracy: 0.2215 - loss: 1.3854 - val_accuracy: 0.2571 - val_loss: 1.3863
Epoch 4/50
356/356 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.2500 - loss: 1.3969 - val_accuracy: 0.2387 - val_loss: 1.3863
Epoch 5/50
356/356 ━━━━━━━━━━━━━━━━━━━━ 34s 95ms/step - accuracy: 0.2408 - loss: 1.3873 - val_accuracy: 0.2319 - val_loss: 1.3863
Epoch 6/50
356/356 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.2500 - loss: 1.3411 - val_accuracy: 0.2624 - val_loss: 1.3863
Epoch 7/50
356/356 ━━━━━━━━━━━━━━━━━━━━ 34s 95ms/step - accuracy: 0.2313 - loss: 1.3870 - val_accuracy: 0.2540 - val_loss: 1.3863
Epoch 8/50
356/356 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3125 - loss: 1.4329 - val_accuracy: 0.2372 - val_loss: 1.3863
Epoch 9/50
356/356 ━━━━━━━━━━━━━━━━━━━━ 34s 95ms/step - accuracy: 0.2487 - loss: 1.3858 - val_accuracy: 0.231

### Save the model

In [14]:
model.save("capsnet_brain_tumor_model.h5")

In [15]:
model.save('capsnet_brain_tumor_model.keras')

### Evaluate the model

In [16]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 1.386294960975647
Test accuracy: 0.22807016968727112


### Generate classification report

In [17]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes, target_names=categories))

41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step
              precision    recall  f1-score   support

      glioma       0.22      0.59      0.32       300
  meningioma       0.22      0.22      0.22       306
     notumor       0.27      0.08      0.12       405
   pituitary       0.25      0.06      0.09       300

    accuracy                           0.23      1311
   macro avg       0.24      0.24      0.19      1311
weighted avg       0.24      0.23      0.18      1311



### Confusion matrix

In [18]:
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print(conf_matrix)

[[178  76  26  20]
 [189  68  34  15]
 [269  88  33  15]
 [178  74  31  17]]
